This notebook downloads a small piece of a text dataset to local cache (`/Users/USER/.cache/huggingface/datasets` on macbook). Then, you should manually install the files (which are dicts for activations and locations for each feature) from [https://huggingface.co/datasets/MrGonao/raw_features_gemma16/tree/main](https://huggingface.co/datasets/MrGonao/raw_features_gemma16/tree/main) to the features folder. After doing this, the code below will find you the activating documents for each feature. 

In [4]:
#!pip install transformers transformer_lens datasets utils
from datasets import load_dataset
from transformer_lens import utils
from safetensors.numpy import load_file
from transformers import AutoTokenizer
from nnsight import LanguageModel
from datasets import Dataset
from safetensors.numpy import load_file
import numpy as np
import torch
tokenizer = LanguageModel("google/gemma-2-9B").tokenizer
data = load_dataset("kh4dien/fineweb-100m-sample", name="", split="train")
tokens = utils.tokenize_and_concatenate(data, tokenizer, max_length=256,column_name="text")
tokens = tokens.shuffle(22)["tokens"]
feature_dict = load_file("features/11_0_3275.safetensors")
activations,locations = feature_dict['activations'], feature_dict['locations']
locations = torch.tensor(locations.astype(np.int64))
def view_batch(n: int):
    if n >= len(tokens):
        return None
    doc = tokens[n]
    return tokenizer.batch_decode(doc)

#select locations where 10th feature activates.
def visualize_feature(n, activations, locations):
    print(f"Visualizing Feature {n}")
    idx = locations[:,2]== n
    locations = locations[idx]
    activations = activations[idx]
    location_dicts = []
    for location, activation in zip(locations, activations):
        d = {}
        d['batch'] = location[0]
        d['position'] = location[1]
        d['feature'] = location[2]
        d['activation'] = activation
        location_dicts.append(d)
    sorted_location_dicts = sorted(location_dicts, key=lambda x: x['activation'], reverse=True)
    #This now has batch where activated & token position where activated
    count = 0
    for d in sorted_location_dicts:
        batch = view_batch(int(d['batch']))
        if batch is None:
            pass
        else:
            count += 1
            pos = d['position']
            context = ''.join([s for s in batch[pos-10:pos+10] if s != '\n'])
            print(f'{batch[pos]:<15} @ {context:<50} {d["activation"]}')
        if count > 10:
            break

print(len(locations))
visualize_feature(25, activations, locations)

142384861
Visualizing Feature 25
 photons        @  in which it bends about the middle.Infrared photons will be absorbed by CO2 in that band 71.8125
 photons        @  decay, falling back into the ground state and emitting photons in the process. The warmer the atmosphere the 63.0
 radiation      @  level, they emitted much shorter pulses than the electromagnetic radiation going in — just a few femtoseconds 58.40625
 electromagnetic @  energy level, they emitted much shorter pulses than the electromagnetic radiation going in — just a few femto 57.625
 radiation      @  is the complete interval of all possible frequencies of electromagnetic radiation. It is the air space through which data 56.5625
 radiation      @  sample.But a vibrating molecule will also emit radiation at wavelength corresponding to the energy of the vibration 56.5625
 electromagnetic @ rafast THz pulses (the part of the electromagnetic spectrum between microwaves and infrared light). That 55.375
 photons        @ . 